# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

* Импортируем необходимые библиотеки и их функции
* Просмотрим все файлы

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import stats as st
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

Проверим данные на дубликаты и пропуски

In [3]:
print(data0.duplicated().sum())
print(data1.duplicated().sum())
print(data2.duplicated().sum())

0
0
0


In [4]:
print(data0.isna().sum())
print(data1.isna().sum())
print(data2.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Дубликатов и пропусков не обнаружено

In [5]:
print('Регион 0:', data0.head(), '\n')
print('Регион 1:', data1.head(), '\n')
print('Регион 2:', data2.head())

Регион 0:       id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647 

Регион 1:       id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305 

Регион 2:       id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


Уберём столбец 'id', так как он не понадобится для дальнейшей работы

In [6]:
data0 = data0.drop(columns=['id'], axis=0)
data1 = data1.drop(columns=['id'], axis=0)
data2 = data2.drop(columns=['id'], axis=0)

Разделим данные на тренировочные и валидационные выборки и отделим целевой признак

In [7]:
def data_split(data):
    train, valid = train_test_split(data, test_size=0.25, random_state=42)
    features_train = train.drop(columns=['product'], axis=0)
    target_train = train['product']
    features_valid = valid.drop(columns=['product'], axis=0)
    target_valid = valid['product']
    return features_train, target_train, features_valid, target_valid

In [8]:
features_train_0, target_train_0, features_valid_0, target_valid_0 = data_split(data0)
features_train_1, target_train_1, features_valid_1, target_valid_1 = data_split(data1)
features_train_2, target_train_2, features_valid_2, target_valid_2 = data_split(data2)

Масштабируем признаки методом стандартизации данных

In [9]:
scaler = StandardScaler()
scaler.fit(features_train_0)
features_train_0 = scaler.transform(features_train_0)
features_valid_0 = scaler.transform(features_valid_0)

scaler.fit(features_train_1)
features_train_1 = scaler.transform(features_train_1)
features_valid_1 = scaler.transform(features_valid_1)

scaler.fit(features_train_2)
features_train_2 = scaler.transform(features_train_2)
features_valid_2 = scaler.transform(features_valid_2)

## Обучение и проверка модели

Для нашей задачи подходит модель Линейной Регрессии, так что будем использовать именно её

In [10]:
model = LinearRegression()

* Разделим данные на выборки и обучим модель
* Сохраним предсказания в переменные predict

In [11]:
def prediction(features_train, target_train, features_valid, target_valid):
    model.fit(features_train, target_train)
    predict = pd.Series(model.predict(features_valid), target_valid.index)
    return predict

In [12]:
predict0 = prediction(features_train_0, target_train_0, features_valid_0, target_valid_0)
predict1 = prediction(features_train_1, target_train_1, features_valid_1, target_valid_1)
predict2 = prediction(features_train_2, target_train_2, features_valid_2, target_valid_2)

In [13]:
predictions = [predict0, predict1, predict2]
target_valids = [target_valid_0, target_valid_1, target_valid_2]

Посчитаем средние запасы нефти в каждом регионе и RMSE

In [14]:
for i in range(len(predictions)):
    print('Регион', i)
    mse = mean_squared_error(predictions[i], target_valids[i])
    print('Средний запас: {:.2f}'.format(predictions[i].mean()))
    rmse = mse ** 0.5
    print('RMSE: {:.2f}'.format(rmse), '\n')

Регион 0
Средний запас: 92.40
RMSE: 37.76 

Регион 1
Средний запас: 68.71
RMSE: 0.89 

Регион 2
Средний запас: 94.77
RMSE: 40.15 



* Как видим, для Региона 1 разница между прогнозируемыми и наблюдаемыми значениями наименьшая. 
* В Регионах 0 и 2 модель сильно ошибается. 

## Подготовка к расчёту прибыли

* Создадим новые переменные для дальнейшей работы с необходимыми данными
* Посчитаем стоимость 1 скважины и объём окупаемости

In [15]:
BUDGET = 10 ** 10 #бюджет
WELLS = 200 #количество нефтяных вышек
BARREL_PRICE = 450000 #стоимоть 1000 баррелей нефти
RISK = 0.025 
WELL_PRICE = BUDGET / WELLS #стоимость 1 скважины
PAYBACK_VOLUME = WELL_PRICE / BARREL_PRICE #объём окупаемости
print('Стоимость 1 скважины:', WELL_PRICE, 'рублей')
print('Объём окупаемости: {:.2f}'.format(PAYBACK_VOLUME), 'баррелей')

Стоимость 1 скважины: 50000000.0 рублей
Объём окупаемости: 111.11 баррелей


Делаем вывод, что:
* 1 скважина стоит 50.000.000 рублей
* Объём окупаемости составляет 111 баррелей

## Расчёт прибыли и рисков 

In [16]:
def profits(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return selected.sum() * BARREL_PRICE - BUDGET

In [17]:
state = np.random.RandomState(42)
for i in range(len(target_valids)):
    values = []
    
    for _ in range(1000):
        target_subsample = target_valids[i].sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[i][target_subsample.index] 
        profit = profits(target_subsample, probs_subsample, WELLS)
        values.append(profit)
        
    name = 'Регион: ' + str(i)
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    
    print(name)
    print('Средняя выручка: {:.2f}'.format(mean * 1000))
    print ('95% доверительный интервал: {:.2f}, {:.2f}'.format(lower, upper))
    print('Риск убытков: {:.2%}'.format((values < 0).mean()), '\n')

Регион: 0
Средняя выручка: 427847560462.52
95% доверительный интервал: -97249829.57, 954215192.71
Риск убытков: 5.50% 

Регион: 1
Средняя выручка: 511530218148.52
95% доверительный интервал: 91700564.14, 921455668.33
Риск убытков: 0.60% 

Регион: 2
Средняя выручка: 408545681214.94
95% доверительный интервал: -120624872.94, 960859440.73
Риск убытков: 7.50% 



# Вывод:
* Условиям подходит только 1 регион - 'Регион 1'
* Средняя выручка составляет - 511 530 218 148.52 (более 500 миллиардов) рублей 
* 95% доверительный интервал составляет: 
* Нижняя граница - 91 700 564.14 
* Верхняя граница - 921 455 668.33
* Риск убытков составляет - 0.6%

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован